In [2]:
import json
import math
import conllu
import numpy as np
from tqdm import tqdm
from conllu import parse, parse_incr, parse_sentences

In [2]:
corpus_file = json.load(open('../corpus.json','r', encoding="utf-8")) 
print(len(corpus_file))
print(corpus_file[0])

26902
[{'form': '2012年', 'upos': 'NOUN'}, {'form': '12月', 'upos': 'NOUN'}, {'form': '我', 'upos': 'PRON'}, {'form': '在', 'upos': 'ADP'}, {'form': '韩国', 'upos': 'PROPN'}, {'form': '留学', 'upos': 'VERB'}, {'form': '的', 'upos': 'PART'}, {'form': '时候', 'upos': 'NOUN'}, {'form': '，', 'upos': 'PUNCT'}, {'form': '有', 'upos': 'VERB'}, {'form': '一', 'upos': 'NUM'}, {'form': '天', 'upos': 'NOUN'}, {'form': '接', 'upos': 'VERB'}, {'form': '到', 'upos': 'VERB'}, {'form': '一', 'upos': 'NUM'}, {'form': '个', 'upos': 'NOUN'}, {'form': '通知', 'upos': 'NOUN'}, {'form': '。', 'upos': 'PUNCT'}]


# 1. 概率方法
## step1  统计数据

In [5]:
# 频率方法
frequency_bigram_set = {}                  # 双层字典
frequency_bigram_set2 = {}                 # 单层字典
for sentence in tqdm(corpus_file):
    for word_index in range(len(sentence) - 1):
        if sentence[word_index]['upos'] == 'PUNCT' or sentence[word_index + 1]['upos'] == 'PUNCT' or (sentence[word_index]['form'] in ['。', '，', '“', '”','——']) or (sentence[word_index + 1]['form'] in ['。', '，', '“', '”','——']):
            continue

        frequency_bigram_set2[f"{sentence[word_index]['form']}&{sentence[word_index]['upos']}&{sentence[word_index + 1]['form']}&{sentence[word_index + 1]['upos']}"] = 0
        try:
            frequency_bigram_set[f"{sentence[word_index]['form']}&{sentence[word_index]['upos']}"][f"{sentence[word_index + 1]['form']}&{sentence[word_index + 1]['upos']}"] = 0
        except:
            frequency_bigram_set[f"{sentence[word_index]['form']}&{sentence[word_index]['upos']}"] = {}
            frequency_bigram_set[f"{sentence[word_index]['form']}&{sentence[word_index]['upos']}"][f"{sentence[word_index + 1]['form']}&{sentence[word_index + 1]['upos']}"] = 0

for sentence in tqdm(corpus_file):
    for word_index in range(len(sentence) - 1):
        if sentence[word_index]['upos'] == 'PUNCT' or sentence[word_index + 1]['upos'] == 'PUNCT' or (sentence[word_index]['form'] in ['。', '，', '“', '”','——']) or (sentence[word_index + 1]['form'] in ['。', '，', '“', '”','——']):
            continue
        
        frequency_bigram_set2[f"{sentence[word_index]['form']}&{sentence[word_index]['upos']}&{sentence[word_index + 1]['form']}&{sentence[word_index + 1]['upos']}"] += 1
        frequency_bigram_set[f"{sentence[word_index]['form']}&{sentence[word_index]['upos']}"][f"{sentence[word_index + 1]['form']}&{sentence[word_index + 1]['upos']}"] += 1

100%|██████████| 26902/26902 [00:01<00:00, 16394.64it/s]


In [6]:
# 数据保存
json.dump(frequency_bigram_set, open('../output/概率方法/frequency_bigram_set.json','w', encoding ="utf-8"), ensure_ascii = False, indent = 2)
json.dump(frequency_bigram_set2, open('../output/概率方法/frequency_bigram_set2.json','w', encoding ="utf-8"), ensure_ascii = False, indent = 2)

## step2  分析结果

In [3]:
# 频率方法 排序
frequency_bigram_set = json.load(open('../output/概率方法/frequency_bigram_set.json','r', encoding="utf-8")) 
frequency_bigram_set2 = json.load(open('../output/概率方法/frequency_bigram_set2.json','r', encoding="utf-8")) 
frequency_bigram_set2_sorted = sorted(frequency_bigram_set2.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)

In [4]:
# 查询某个类别
def frequencyByTagPattern(frequency_bigram_set2_sorted, tag_pattern, K = 10):
    print(f'===================== frequency By Tag Pattern: ({tag_pattern[0]} {tag_pattern[1]}) =====================')
    count = 0
    for bigram in frequency_bigram_set2_sorted:
        if count >= K:
            break
        pattern0 = bigram[0].split('&')[1]
        pattern1 = bigram[0].split('&')[3]
        if (not tag_pattern[0] == None) and (not tag_pattern[1] == None) and pattern0 == tag_pattern[0] and pattern1 == tag_pattern[1]:
            count += 1
            print(f'{count}: {bigram}')
        elif (not tag_pattern[0] == None) and (tag_pattern[1] == None) and pattern0 == tag_pattern[0]:
            count += 1
            print(f'{count}: {bigram}')
        elif (tag_pattern[0] == None) and (not tag_pattern[1] == None) and pattern1 == tag_pattern[1]:
            count += 1
            print(f'{count}: {bigram}')
        elif (tag_pattern[0] == None) and (tag_pattern[1] == None):
            count += 1
            print(f'{count}: {bigram}')

def frequencyByWord0(frequency_bigram_set, word0, tag_pattern0, tag_pattern1 = None, K = 10):
    print(f'===================== frequency By Word0: ({word0}&{tag_pattern0} {tag_pattern1}) =====================')
    sub_frequency_bigram_set = frequency_bigram_set[f'{word0}&{tag_pattern0}']
    sub_frequency_bigram_set = sorted(sub_frequency_bigram_set.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)
    count = 0
    for word1_pattern1 in sub_frequency_bigram_set:
        if count >= K:
            break
        
        pattern1 = word1_pattern1[0].split('&')[1]
        if (not tag_pattern1 == None) and pattern1 == tag_pattern1:
            count += 1
            print(f'{count}: ({word0}&{tag_pattern0}&{word1_pattern1[0]}, {word1_pattern1[1]})')
        elif tag_pattern1 == None:
            count += 1
            print(f'{count}: ({word0}&{tag_pattern0}&{word1_pattern1[0]}, {word1_pattern1[1]})')

# 按照可能性大小得顺序列出十个搭配及相关分数
tag_pattern = ['VERB', 'NOUN']
frequencyByTagPattern(frequency_bigram_set2_sorted, tag_pattern)
frequencyByTagPattern(frequency_bigram_set2_sorted, [None, None])

frequencyByWord0(frequency_bigram_set, '高举', 'VERB', tag_pattern1 = None)
frequencyByWord0(frequency_bigram_set, '高举', 'VERB', tag_pattern1 = 'NOUN')

===================== frequency By Tag Pattern: (VERB NOUN) =====================
1: ('附&VERB&图片&NOUN', 293)
2: ('令&VERB&人&NOUN', 144)
3: ('反&VERB&腐败&NOUN', 139)
4: ('是&VERB&我国&NOUN', 93)
5: ('高举&VERB&邓小平理论&NOUN', 83)
6: ('为&VERB&核心&NOUN', 83)
7: ('跨&VERB&世纪&NOUN', 78)
8: ('让&VERB&人&NOUN', 73)
9: ('是&VERB&个&NOUN', 70)
10: ('有&VERB&可能&NOUN', 69)
===================== frequency By Tag Pattern: (None None) =====================
1: ('的&PART&一&NUM', 1047)
2: ('新&ADJ&的&PART', 768)
3: ('这&PRON&一&NUM', 645)
4: ('一&NUM&种&NOUN', 626)
5: ('这&PRON&是&VERB', 584)
6: ('了&PART&一&NUM', 577)
7: ('的&PART&发展&NOUN', 544)
8: ('一&NUM&年&NOUN', 544)
9: ('中&ADP&的&PART', 522)
10: ('就&ADV&是&VERB', 520)
===================== frequency By Word0: (高举&VERB None) =====================
1: (高举&VERB&邓小平理论&NOUN, 83)
2: (高举&VERB&旗帜&NOUN, 4)
3: (高举&VERB&毛泽东思想&NOUN, 3)
4: (高举&VERB&邓&PROPN, 2)
5: (高举&VERB&伟大&ADJ, 2)
6: (高举&VERB&起&VERB, 1)
7: (高举&VERB&胳膊&NOUN, 1)
8: (高举&VERB&社会主义&NOUN, 1)
9: (高举&VERB&爱国主义&NOUN, 1)
10: (高举&VERB

# 2. t-test 假设检验方法
## step1  统计数据

In [15]:
# t-test 假设检验方法
frequency_unigram_set = {}                  # 单层字典，统计每个词出现的个数
for sentence in tqdm(corpus_file):
    for word_index in range(len(sentence)):
        if sentence[word_index]['upos'] == 'PUNCT' or sentence[word_index]['form'] in ['。', '，', '“', '”','——']:
            continue
        try:
            frequency_unigram_set[f"{sentence[word_index]['form']}&{sentence[word_index]['upos']}"] += 1
        except:
            frequency_unigram_set[f"{sentence[word_index]['form']}&{sentence[word_index]['upos']}"] = 1
json.dump(frequency_unigram_set, open('../output/t-test假设检验方法/frequency_unigram_set.json','w', encoding ="utf-8"), ensure_ascii = False, indent = 2)

100%|██████████| 26902/26902 [00:00<00:00, 42263.72it/s]


In [16]:
# t-test 假设检验方法
frequency_unigram_set = json.load(open('../output/t-test假设检验方法/frequency_unigram_set.json','r', encoding="utf-8")) 
frequency_bigram_set2 = json.load(open('../output/概率方法/frequency_bigram_set2.json','r', encoding="utf-8")) 
N_unigram = 1087497             # 没有标点符号的 unigram
N_bigram = 899680               # 没有标点符号的 bigram

1087497


In [19]:
# 计算每个搭配的t
t_set = {}

for bigram in tqdm(frequency_bigram_set2):
    word_pattern = bigram.split('&')
    word0 = f'{word_pattern[0]}&{word_pattern[1]}'
    word1 = f'{word_pattern[2]}&{word_pattern[3]}'
    x_ = frequency_bigram_set2[bigram] / N_bigram
    mu = (frequency_unigram_set[word0] / N_bigram) * (frequency_unigram_set[word1] / N_bigram)
    t = (x_ - mu) / math.sqrt(x_ / N_bigram)

    t_set[bigram] = t
json.dump(t_set, open('../output/t-test假设检验方法/t_set.json','w', encoding ="utf-8"), ensure_ascii = False, indent = 2)

100%|██████████| 456721/456721 [00:00<00:00, 560511.06it/s]


## step2  分析结果

In [6]:
# 频率方法 排序
frequency_unigram_set = json.load(open('../output/t-test假设检验方法/frequency_unigram_set.json','r', encoding="utf-8"))
frequency_bigram_set = json.load(open('../output/概率方法/frequency_bigram_set.json','r', encoding="utf-8"))
frequency_bigram_set2 = json.load(open('../output/概率方法/frequency_bigram_set2.json','r', encoding="utf-8"))
t_set = json.load(open('../output/t-test假设检验方法/t_set.json','r', encoding="utf-8"))
t_set_sorted = sorted(t_set.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)

In [21]:
def getUnigramFrequency(frequency_unigram_set, word, tag_pattern = None):
    if (not word == None) and (not tag_pattern == None):
        try:
            return frequency_unigram_set[f'{word}&{tag_pattern}']
        except:
            return 0
    elif (not word == None) and (tag_pattern == None):
        f = 0
        for unigram in frequency_unigram_set:
            word_pattern = bigram.split('&')
            if word == word_pattern[0]:
                f += frequency_unigram_set[unigram]
        return f
    elif (word == None) and (not tag_pattern == None):
        f = 0
        for bigram in frequency_unigram_set:
            word_pattern = bigram.split('&')
            if tag_pattern == word_pattern[1]:
                f += frequency_unigram_set[bigram]
        return f
    elif (word == None) and (tag_pattern == None):
        return 0

def getBigramFrequency(frequency_bigram_set2, word0, word1, tag_pattern0 = None, tag_pattern1 = None):
    if (word0 == None) and (tag_pattern0 == None) and (word1 == None) and (tag_pattern1 == None):
        return 0
    elif (not tag_pattern0 == None) and (not tag_pattern1 == None):
        try:
            return frequency_bigram_set2[f'{word0}&{tag_pattern0}&{word1}&{tag_pattern1}']
        except:
            return 0
    elif (not tag_pattern0 == None) and (tag_pattern1 == None):
        f = 0
        for bigram in frequency_bigram_set2:
            word_pattern = bigram.split('&')
            if word0 == word_pattern[0] and tag_pattern0 == word_pattern[1] and word1 == word_pattern[2]:
                f += frequency_bigram_set2[bigram]
        return f
    elif (tag_pattern0 == None) and (not tag_pattern1 == None):
        f = 0
        for bigram in frequency_bigram_set2:
            word_pattern = bigram.split('&')
            if word0 == word_pattern[0] and word1 == word_pattern[2] and tag_pattern1 == word_pattern[3]:
                f += frequency_bigram_set2[bigram]
        return f
    elif (tag_pattern0 == None) and (tag_pattern1 == None):
        f = 0
        for bigram in frequency_bigram_set2:
            word_pattern = bigram.split('&')
            if word0 == word_pattern[0] and word1 == word_pattern[2]:
                f += frequency_bigram_set2[bigram]
        return f

def calT(frequency_bigram_set2, word01, word02, word1, tag_pattern01 = None, tag_pattern02 = None, tag_pattern1 = None):
    f1 = getBigramFrequency(frequency_bigram_set2, word01, word1, tag_pattern01, tag_pattern1)
    f2 = getBigramFrequency(frequency_bigram_set2, word02, word1, tag_pattern02, tag_pattern1)
    t = (f1 - f2) / math.sqrt(f1 + f2)
    return t

def tOfDifferences(t_set_sorted, frequency_bigram_set, frequency_bigram_set2, word01, word02, tag_pattern01, tag_pattern02, K = 10):
    print(f'===================== t of Differences: ({word01}-{tag_pattern01} {word02}-{tag_pattern02}) =====================')
    if word01 == None and word02 == None and tag_pattern01 == None and tag_pattern02 == None:
        return 0

    # 统计出现过的后面的词
    word1_list = []
    for word_pattern in frequency_bigram_set[f'{word01}&{tag_pattern01}']:
        word1_list.append(word_pattern)
    for word_pattern in frequency_bigram_set[f'{word02}&{tag_pattern02}']:
        word1_list.append(word_pattern)
    word1_list = list(set(word1_list))      # 去重

    t_result = {}
    for word_pattern in word1_list:
        word1 = word_pattern.split('&')[0]
        pattern1 = word_pattern.split('&')[1]
        t_result[word_pattern] = calT(frequency_bigram_set2, word01, word02, word1, tag_pattern01, tag_pattern02, tag_pattern1 = pattern1)


    t_result = sorted(t_result.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)
    for i in range(1, K + 1):
        bigram = t_result[i]
        word_pattern = bigram[0].split('&')
        word1 = word_pattern[0]
        pattern1 = word_pattern[1]
        f0 = getUnigramFrequency(frequency_unigram_set, word = word1, tag_pattern = pattern1)
        f1 = getBigramFrequency(frequency_bigram_set2, word01, word1, tag_pattern01, pattern1)
        f2 = getBigramFrequency(frequency_bigram_set2, word02, word1, tag_pattern02, pattern1)
        print(f'{i}: {bigram}, c({word1}-{pattern1}) = {f0}, c({word01}-{tag_pattern01}-{word1}-{pattern1}) = {f1}, c({word02}-{tag_pattern02}-{word1}-{pattern1}) = {f2}')
    print('-------------------------------------------')
    for i in range(-1, -K-1, -1):
        bigram = t_result[i]
        word_pattern = bigram[0].split('&')
        word1 = word_pattern[0]
        pattern1 = word_pattern[1]
        f0 = getUnigramFrequency(frequency_unigram_set, word = word1, tag_pattern = pattern1)
        f1 = getBigramFrequency(frequency_bigram_set2, word01, word1, tag_pattern01, pattern1)
        f2 = getBigramFrequency(frequency_bigram_set2, word02, word1, tag_pattern02, pattern1)
        print(f'{-i}: ({bigram[0]}, {-bigram[1]}), c({word1}-{pattern1}) = {f0}, c({word01}-{tag_pattern01}-{word1}-{pattern1}) = {f1}, c({word02}-{tag_pattern02}-{word1}-{pattern1}) = {f2}')
    

def tByTagPattern(t_set_sorted, frequency_unigram_set, frequency_bigram_set2, tag_pattern, K = 10):
    print(f'===================== t By Tag Pattern: ({tag_pattern[0]} {tag_pattern[1]}) =====================')
    count = 0
    for bigram in t_set_sorted:
        if count >= K:
            break
        word_pattern = bigram[0].split('&')
        word0 = word_pattern[0]
        pattern0 = word_pattern[1]
        word1 = word_pattern[2]
        pattern1 = word_pattern[3]
        if (not tag_pattern[0] == None) and (not tag_pattern[1] == None) and pattern0 == tag_pattern[0] and pattern1 == tag_pattern[1]:
            count += 1
            f0 = getUnigramFrequency(frequency_unigram_set, word = word0, tag_pattern = pattern0)
            f1 = getUnigramFrequency(frequency_unigram_set, word = word1, tag_pattern = pattern1)
            f2 = getBigramFrequency(frequency_bigram_set2, word0, word1, pattern0, pattern1)
            print(f'{count}: {bigram}, c({word0}-{pattern0}) = {f0}, c({word1}-{pattern1}) = {f1}, c({bigram[0]}) = {f2}')
        elif (not tag_pattern[0] == None) and (tag_pattern[1] == None) and pattern0 == tag_pattern[0]:
            count += 1
            f0 = getUnigramFrequency(frequency_unigram_set, word = word0, tag_pattern = pattern0)
            f1 = getUnigramFrequency(frequency_unigram_set, word = word1, tag_pattern = pattern1)
            f2 = getBigramFrequency(frequency_bigram_set2, word0, word1, pattern0, pattern1)
            print(f'{count}: {bigram}, c({word0}-{pattern0}) = {f0}, c({word1}-{pattern1}) = {f1}, c({bigram[0]}) = {f2}')
        elif (tag_pattern[0] == None) and (not tag_pattern[1] == None) and pattern1 == tag_pattern[1]:
            count += 1
            f0 = getUnigramFrequency(frequency_unigram_set, word = word0, tag_pattern = pattern0)
            f1 = getUnigramFrequency(frequency_unigram_set, word = word1, tag_pattern = pattern1)
            f2 = getBigramFrequency(frequency_bigram_set2, word0, word1, pattern0, pattern1)
            print(f'{count}: {bigram}, c({word0}-{pattern0}) = {f0}, c({word1}-{pattern1}) = {f1}, c({bigram[0]}) = {f2}')
        elif (tag_pattern[0] == None) and (tag_pattern[1] == None):
            count += 1
            f0 = getUnigramFrequency(frequency_unigram_set, word = word0, tag_pattern = pattern0)
            f1 = getUnigramFrequency(frequency_unigram_set, word = word1, tag_pattern = pattern1)
            f2 = getBigramFrequency(frequency_bigram_set2, word0, word1, pattern0, pattern1)
            print(f'{count}: {bigram}, c({word0}-{pattern0}) = {f0}, c({word1}-{pattern1}) = {f1}, c({bigram[0]}) = {f2}')

def tByWord0(t_set_sorted, frequency_unigram_set, frequency_bigram_set2, word0, tag_pattern0, tag_pattern1 = None, K = 10):
    print(f'===================== t By Word0: ({word0}&{tag_pattern0} {tag_pattern1}) =====================')
    if word0 == None:
        return 0

    count = 0
    for bigram in t_set_sorted:
        if count >= K:
            break
        word_pattern = bigram[0].split('&')
        pattern0 = word_pattern[1]
        word1 = word_pattern[2]
        pattern1 = word_pattern[3]
        if (not tag_pattern[0] == None) and (not tag_pattern[1] == None) and pattern0 == tag_pattern[0] and pattern1 == tag_pattern[1] and word0 == word_pattern[0]:
            count += 1
            f0 = getUnigramFrequency(frequency_unigram_set, word = word0, tag_pattern = pattern0)
            f1 = getUnigramFrequency(frequency_unigram_set, word = word1, tag_pattern = pattern1)
            f2 = getBigramFrequency(frequency_bigram_set2, word0, word1, pattern0, pattern1)
            print(f'{count}: {bigram}, c({word0}-{pattern0}) = {f0}, c({word1}-{pattern1}) = {f1}, c({bigram[0]}) = {f2}')
        elif (not tag_pattern[0] == None) and (tag_pattern[1] == None) and pattern0 == tag_pattern[0] and word0 == word_pattern[0]:
            count += 1
            f0 = getUnigramFrequency(frequency_unigram_set, word = word0, tag_pattern = pattern0)
            f1 = getUnigramFrequency(frequency_unigram_set, word = word1, tag_pattern = pattern1)
            f2 = getBigramFrequency(frequency_bigram_set2, word0, word1, pattern0, pattern1)
            print(f'{count}: {bigram}, c({word0}-{pattern0}) = {f0}, c({word1}-{pattern1}) = {f1}, c({bigram[0]}) = {f2}')
        elif (tag_pattern[0] == None) and (not tag_pattern[1] == None) and pattern1 == tag_pattern[1] and word0 == word_pattern[0]:
            count += 1
            f0 = getUnigramFrequency(frequency_unigram_set, word = word0, tag_pattern = pattern0)
            f1 = getUnigramFrequency(frequency_unigram_set, word = word1, tag_pattern = pattern1)
            f2 = getBigramFrequency(frequency_bigram_set2, word0, word1, pattern0, pattern1)
            print(f'{count}: {bigram}, c({word0}-{pattern0}) = {f0}, c({word1}-{pattern1}) = {f1}, c({bigram[0]}) = {f2}')
        elif (tag_pattern[0] == None) and (tag_pattern[1] == None) and word0 == word_pattern[0]:
            count += 1
            f0 = getUnigramFrequency(frequency_unigram_set, word = word0, tag_pattern = pattern0)
            f1 = getUnigramFrequency(frequency_unigram_set, word = word1, tag_pattern = pattern1)
            f2 = getBigramFrequency(frequency_bigram_set2, word0, word1, pattern0, pattern1)
            print(f'{count}: {bigram}, c({word0}-{pattern0}) = {f0}, c({word1}-{pattern1}) = {f1}, c({bigram[0]}) = {f2}')

In [22]:
tag_pattern = ['VERB', 'NOUN']
tByTagPattern(t_set_sorted, frequency_unigram_set, frequency_bigram_set2, tag_pattern)
tByTagPattern(t_set_sorted, frequency_unigram_set, frequency_bigram_set2, [None, None])

tByWord0(t_set_sorted, frequency_unigram_set, frequency_bigram_set2, '高举', 'VERB', tag_pattern1 = None)
tByWord0(t_set_sorted, frequency_unigram_set, frequency_bigram_set2, '高举', 'VERB', tag_pattern1 = 'NOUN')

tOfDifferences(t_set_sorted, frequency_bigram_set, frequency_bigram_set2, word01 = '美好', word02 = '热情', tag_pattern01 = 'ADJ', tag_pattern02 = 'ADJ', K = 10)

===================== t By Tag Pattern: (VERB NOUN) =====================
1: ('附&VERB&图片&NOUN', 17.105212930501057), c(附-VERB) = 295, c(图片-NOUN) = 628, c(附&VERB&图片&NOUN) = 293
2: ('令&VERB&人&NOUN', 11.926201538324738), c(令-VERB) = 245, c(人-NOUN) = 3252, c(令&VERB&人&NOUN) = 144
3: ('反&VERB&腐败&NOUN', 11.784244751284598), c(反-VERB) = 286, c(腐败-NOUN) = 207, c(反&VERB&腐败&NOUN) = 139
4: ('高举&VERB&邓小平理论&NOUN', 9.10737189709686), c(高举-VERB) = 105, c(邓小平理论-NOUN) = 239, c(高举&VERB&邓小平理论&NOUN) = 83
5: ('为&VERB&核心&NOUN', 9.060104016019359), c(为-VERB) = 2845, c(核心-NOUN) = 145, c(为&VERB&核心&NOUN) = 83
6: ('跨&VERB&世纪&NOUN', 8.823086804689563), c(跨-VERB) = 126, c(世纪-NOUN) = 547, c(跨&VERB&世纪&NOUN) = 78
7: ('让&VERB&人&NOUN', 8.270284507909636), c(让-VERB) = 647, c(人-NOUN) = 3252, c(让&VERB&人&NOUN) = 73
8: ('有&VERB&可能&NOUN', 8.241930650705863), c(有-VERB) = 5494, c(可能-NOUN) = 88, c(有&VERB&可能&NOUN) = 69
9: ('是&VERB&我国&NOUN', 8.219718808816781), c(是-VERB) = 10347, c(我国-NOUN) = 1194, c(是&VERB&我国&NOUN) = 93
10: ('送&V